In [7]:
import numpy as np
from common.functions import softmax
from common.other_class import Rnnlm, BetterRnnlm

class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]
        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())
            
            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))
        return word_ids
    def get_state(self):
        return self.lstm_layer.h, self.lstm_layer.c
    def set_state(self, state):
        self.lstm_layer.set_state(*state)

### None Training

In [9]:
import ptb

model = RnnlmGen()

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[word] for word in skip_words]

word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '\n')
print(txt)

you doyle mundane smallest kirk seeks margins homes n't polyethylene cell kia improvements mo. grade exempt suez dec. swung torrijos concord spate keating structures crossing willingness rape principles obliged returned pumped nearly evolution effectiveness aggregates rockefeller schemes day-to-day willful cs gartner formal vaccine centers eager files czechoslovakia corporations anything joined killing triple-a quotas curbing plug jerry hazards crack cycling hutton reiterated disposal stone fleischmann exception foot conclude riders affecting gathered accumulation murdoch ddb losing owner functions fla waste climb endless longer-term crystals defendants pressured dealers d.c. invitation disabilities claimed ashland strategists arab defaulted recreational appliances unified chlorofluorocarbons venture-capital activist d'arcy


In [11]:
model.reset_state()

In [13]:
model.load_params('Rnnlm.pkl')

start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[word] for word in skip_words]

word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '\n')
print(txt)

you pointed out of the dire mr. roman said
 both california one of the game is n't so low
 surprisingly oct.
 on the american home television institute issued of a moderate buy-back of shares of shares at a team of frankfurt companies
 react to suspend municipal earnings
 perhaps deferring short value or work could appear for borrowing until past particular machines or technology and does n't develop panic much or many manufacturers or control
 very good is a new company hopes that the right reason more high for the market is expected so long


In [14]:
class BetterRnnlmGen(BetterRnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x).flatten()
            p = softmax(score).flatten()

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        states = []
        for layer in self.lstm_layers:
            states.append((layer.h, layer.c))
        return states

    def set_state(self, states):
        for layer, state in zip(self.lstm_layers, states):
            layer.set_state(*state)

In [15]:
from dataset import sequence

(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt', 
                                                         seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

In [1]:
from common.time_layers import *
from common.base_model import BaseModel

class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn
        
        embed_w = (rn(V, D) / 100).astype('f')
        lstm_wx = (rn(D, 4*H) / np.sqrt(D)).astype('f')
        lstm_wh = (rn(H, 4*H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4*H).astype('f')
        
        self.embed = TimeEmbedding(embed_w)
        self.lstm = TimeLSTM(lstm_wx, lstm_wh, lstm_b, stateful=False)
        
        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        self.hs = None
        
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        self.hs = hs
        return hs[:, -1, :]
    
    def backward(self, dh):
        dhs = np.zeros_like(self.hs)
        dhs[:, -1, :] = dh
        
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout